In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pickle
import numpy as np
import torch
import evaluate
import shutil
from ray.tune import CLIReporter

from datasets import load_dataset, Dataset, load_metric
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    set_seed,
    Seq2SeqTrainer,
)

import pandas as pd

from ray import tune
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.integration.torch import TuneReportCallback
import ray.train.huggingface.transformers
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer


/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


# Q2

## prepare and test model

In [2]:
# 加载模型和分词器
model_name = "flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [3]:
input_text = "Where is the capital of China?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> shanghai</s>


/root/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## `train_func()` ---  Encapsulate data preprocessing, training, and evaluation

load & preprocess data

In [4]:
def preprocess_function(examples,mytokenizer):
    # Ensure inputs and targets are lists of strings
    inputs = [str(ex) for ex in examples['input']]
    targets = [str(ex) for ex in examples['output']]

    # Tokenize inputs
    model_inputs = mytokenizer(inputs, max_length=512, truncation=True)
    # Tokenize targets
    # with tokenizer.as_target_tokenizer():
    labels = mytokenizer(targets, max_length=128, truncation=True)
    # Add labels to model inputs
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def save_dataset(dataset, path):
    # dataset.save_to_disk(path)
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'wb') as f:
        pickle.dump(dataset, f)

def load_preprocessed_dataset(path):
    # return Dataset.load_from_disk(path)
    with open(path, 'rb') as f:
        tmp = pickle.load(f)
    return tmp

def load_and_preprocess_datasets(tokenizer=tokenizer):
    train_save_path = 'data/mydata/preprocessed/train.pkl'
    val_save_path = 'data/mydata/preprocessed/val.pkl'
    test_save_path = 'data/mydata/preprocessed/test.pkl'

    if os.path.exists(train_save_path) and os.path.exists(val_save_path) and os.path.exists(test_save_path):
        train_dataset = load_preprocessed_dataset(train_save_path)
        val_dataset = load_preprocessed_dataset(val_save_path)
        test_dataset = load_preprocessed_dataset(test_save_path)
    else:
        train_dataset = load_dataset('csv', data_files='/spark_zc/STA323_zc/proj2/data/mydata/mytrain.csv', cache_dir='data/cachefile')['train']
        val_dataset = load_dataset('csv', data_files='/spark_zc/STA323_zc/proj2/data/mydata/myvalidation.csv', cache_dir='data/cachefile')['train']
        test_dataset = load_dataset('csv', data_files='/spark_zc/STA323_zc/proj2/data/mydata/mytest.csv', cache_dir='data/cachefile')['train']

        train_dataset = train_dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)
        val_dataset = val_dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)
        test_dataset = test_dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)

        save_dataset(train_dataset, train_save_path)
        save_dataset(val_dataset, val_save_path)
        save_dataset(test_dataset, test_save_path)

    return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = load_and_preprocess_datasets()

定义一个训练函数，该函数将接收超参数配置并训练模型。注意要使用Ray的对象存储来获取数据集。

In [5]:
# [1] Encapsulate data preprocessing, training, and evaluation
# logic in a training function
# ============================================================

def train_func(config):
    ###################################* load data
    train_dataset, val_dataset, test_dataset = load_and_preprocess_datasets()

    ###################################* load model and tokenizer
    model_name_path = "/spark_zc/STA323_zc/proj2/flan-t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name_path)
    model = T5ForConditionalGeneration.from_pretrained(model_name_path)

    ###################################* metrix
    def compute_metrics(p):
        predictions, labels = p

        if isinstance(predictions, tuple):
            predictions = predictions[0]
        predictions = np.argmax(predictions, axis=-1)

        predictions = predictions.flatten()
        labels = labels.flatten()

        metric = load_metric("accuracy")
        return metric.compute(predictions=predictions, references=labels)

    training_args = Seq2SeqTrainingArguments(
        output_dir="/spark_zc/STA323_zc/proj2/results",
        learning_rate=config["learning_rate"],
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        num_train_epochs=config["num_epochs"],
        weight_decay=config["weight_decay"],
        save_total_limit=1,          # 只保留一个checkpoint
        load_best_model_at_end=True, # 是否在训练结束时加载最佳模型
        metric_for_best_model="eval_loss", # 选择最佳模型的指标
        greater_is_better=False,  # 表示更小的评估损失表示更好的模型
        save_strategy="epoch",  # 表示每个训练周期结束后保存模型
        evaluation_strategy="epoch", # 每个训练周期结束后评估模型
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[ray.train.huggingface.transformers.RayTrainReportCallback()],
    )

    trainer = ray.train.huggingface.transformers.prepare_trainer(trainer)
    trainer.train()

    # # Save the trained model checkpoint
    # with tune.checkpoint_dir(step=trainer.state.global_step) as checkpoint_dir:
    #     model.save_pretrained(checkpoint_dir)
    #     tokenizer.save_pretrained(checkpoint_dir)

An error `FileNotFoundError: Unable to find '/tmp/ray/session_2024-06-03_23-57-28_281310_151578/artifacts/2024-06-04_00-03-46/TorchTrainer_2024-06-04_00-03-46/working_dirs/TorchTrainer_dba0f_00000_0_2024-06-04_00-03-46/data/mydata/mytrain.csv'` is the direct cause of failure, hence I substitue the relative path with the absolute path.

In [6]:
# # [4] Define a Ray TorchTrainer to launch `train_func` on all workers
# # ===================================================================
# ray_trainer = TorchTrainer(
#     train_func,
#     scaling_config=ScalingConfig(num_workers=1, use_gpu=False),
#     # [4a] If running in a multi-node cluster, this is where you
#     # should configure the run's persistent storage that is accessible
#     # across all worker nodes.
#     # run_config=ray.train.RunConfig(
#     #     failure_config=ray.train.FailureConfig(max_failures = -1)
#     #     )
# )
# result: ray.train.Result = ray_trainer.fit()

# # [5] Load the trained model.
# with result.checkpoint.as_directory() as checkpoint_dir:
#     checkpoint_path = os.path.join(
#         checkpoint_dir,
#         ray.train.huggingface.transformers.RayTrainReportCallback.CHECKPOINT_NAME,
#     )
#     model = T5ForConditionalGeneration.from_pretrained(checkpoint_path)

Access training results

In [7]:
# result.metrics     # The metrics reported during training.
# result.checkpoint  # The latest checkpoint reported during training.
# result.path        # The path where logs are stored.
# result.error       # The exception that was raised, if training failed.

## Search space and init

初始化Ray，并定义调度器以控制试验的调度策略。

In [8]:

scheduler = tune.schedulers.ASHAScheduler(
    metric="eval_accuracy",
    mode="max",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)


In [9]:
def tune_transformer():
    search_space = {
        "learning_rate": tune.loguniform(1e-5, 1e-3),
        "batch_size": tune.choice([8, 16, 32]),
        "num_epochs": tune.choice([1]),
        # "num_epochs": tune.choice([2, 3, 4]),
        "weight_decay": tune.uniform(0.0, 0.3),
    }

    reporter = CLIReporter(
        parameter_columns=["learning_rate", "num_train_epochs", "weight_decay"],
        metric_columns=["eval_accuracy", "eval_loss", "epoch", "training_iteration"],
        max_report_frequency=10,  # 控制报告频率
        print_intermediate_tables=False  # 关闭中间表格输出
    )
    analysis = tune.run(
        train_func,
        resources_per_trial={"cpu": 0.2},  # Adjust as needed
        config=search_space,
        num_samples=1,  # Number of trials
        scheduler=scheduler,
        progress_reporter=reporter,
        name="tune_qa_model",
        local_dir="/spark_zc/STA323_zc/proj2/ray_results",
        stop={"training_iteration": 2},
        keep_checkpoints_num=3,  # 限制保存的 checkpoint 数量
        checkpoint_score_attr="eval_acc",  # 根据验证集的准确率选择保留的 checkpoint
    )

    print("Best hyperparameters found were: ", analysis.best_config)
    best_trial = analysis.get_best_trial(metric="eval_loss", mode="min")
    best_model_path = best_trial.checkpoint.value
    # 创建目标文件夹路径
    destination_path = "/path/to/bestmodel"
    # 复制整个目录
    shutil.copytree(best_model_path, destination_path)

In [10]:
# Run Ray Tune
# ray.shutdown()
ray.init(ignore_reinit_error=True)
tune_transformer()

2024-06-04 02:38:05,081	WARNING services.py:2009 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66101248 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=7.89gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-06-04 02:38:06,282	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/root/anaconda3/lib/python3.11/site-packages/ray/tune/tune.py:583: UserWarning: The `local_dir` argument is deprecated and will be removed. This will pass-through to set the `storage_path` for now but will raise an error in the future. You should only set the `storage_path` from now on.
  warnings.warn(
2024-06-04 02:38:46,079	INFO tune.py:614 -- [output] This uses the legacy output an

== Status ==
Current time: 2024-06-04 02:38:46 (running for 00:00:00.27)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(pid=336911) /root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(pid=336911)   torch.utils._pytree._register_pytree_node(
(pid=336911) /root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(pid=336911)   torch.utils._pytree._register_pytree_node(
Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 10000 examples [00:00, 41070.70 examples/s]
Generating train split: 20000 examples [00:00, 45172.39 examples/s]
Generating train split: 30000 examples [00:00, 50480.11 examples/s]
Generating train split: 40000 examples [00:00, 52764.39 examples/s]
Generating train split: 50000 examples [00:00, 53214.78 examples/s]
Generating train split: 60000 examples 

== Status ==
Current time: 2024-06-04 02:38:56 (running for 00:00:10.37)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0.2/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 5000 examples [00:00, 51092.97 examples/s]
Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 10000 examples [00:00, 73774.41 examples/s]
Generating train split: 20302 examples [00:00, 60754.44 examples/s]
Map:   9%|▊         | 7000/81819 [00:07<01:24, 887.15 examples/s]


...

== Status ==
Current time: 2024-06-04 02:56:30 (running for 00:17:43.89)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0.2/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-06-04 02:56:40 (running for 00:17:53.99)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0.2/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-06-04 02:56:50 (running for 00:18:04.04)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logi

  1%|          | 18/2557 [16:17<35:02:22, 49.68s/it]


== Status ==
Current time: 2024-06-04 02:57:20 (running for 00:18:34.24)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0.2/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-06-04 02:57:30 (running for 00:18:44.34)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0.2/128 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-06-04_02-38-03_004700_328376/artifacts/2024-06-04_02-38-46/tune_qa_model/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-06-04 02:57:40 (running for 00:18:54.37)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logi

RuntimeError: Caught unexpected exception: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.17.0.13, ID: cc6410da2eb894759e0b9d534b66d9fe489099b538bc5897caac4e0d) where the task (actor ID: 996e88f353f9e77e16c7982b01000000, name=ImplicitFunc.__init__, pid=336911, memory used=16.02GB) was running was 30.45GB / 32.00GB (0.951603), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: 06fb3109625fbd86a41e6133fca054b69b0862aa511990a32b1d5887) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.17.0.13`. To see the logs of the worker, use `ray logs worker-06fb3109625fbd86a41e6133fca054b69b0862aa511990a32b1d5887*out -ip 172.17.0.13. Top 10 memory users:
PID	MEM(GB)	COMMAND
336911	16.02	ray::ImplicitFunc.train
127410	2.46	/data/root/.vscode-server/cli/servers/Stable-dc96b837cf6bb4af9cd736aa3af08cf8279f7685/server/node /r...
279474	1.08	/root/anaconda3

(raylet) [2024-06-04 02:58:06,198 E 328649 328649] (raylet) node_manager.cc:3041: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: cc6410da2eb894759e0b9d534b66d9fe489099b538bc5897caac4e0d, IP: 172.17.0.13) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.17.0.13`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [ ]:
# Save the best model
# best_trial = analysis.get_best_trial("eval_loss", "min", "last")
# best_checkpoint_dir = analysis.get_best_checkpoint(best_trial)
# model.save_pretrained(best_checkpoint_dir)
# tokenizer.save_pretrained(best_checkpoint_dir)